In [0]:
import os

import h5py
import numpy as np
from keras import backend as K
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Input, Reshape, Permute, Lambda, TimeDistributed, SpatialDropout1D
from keras.layers.merge import Add
from keras.models import Model
from keras.optimizers import SGD
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
def load_seq_lst(lst_name, num_seq, ovr_num=None, num_joints=25, num_class=52):
    assert (os.path.isfile(lst_name + '.txt') and os.path.isfile(lst_name + '.h5'))
    if ovr_num is None:
        ovr_num = int(num_seq / 2)
    keyname_lst = [item.strip().split() for item in open(lst_name + '.txt', 'r').readlines()]
    X = []
    Y = []
    vid_list = []
    start_list = []
    with h5py.File(lst_name + '.h5', 'r') as hf:
        for item in keyname_lst:
            skeleton = np.asarray(hf.get(item[0]))
            skeleton1 = skeleton[:, 0:75].reshape((skeleton.shape[0], num_joints, 3))
            skeleton2 = skeleton[:, 75:].reshape((skeleton.shape[0], num_joints, 3))
            skeleton = np.concatenate((skeleton1, skeleton2), axis=-1)

            labels = np.asarray(hf.get(item[1]), dtype=int)
            labels_pertime = np.zeros((skeleton.shape[0]), dtype=np.int32)
            for clip_idx in range(len(labels)):
                labels_pertime[labels[clip_idx][1]:labels[clip_idx][2]] = labels[clip_idx][0] + 1
            labels_pertime = labels_pertime.astype(np.int32)
            labels_pertime = np_utils.to_categorical(labels_pertime, num_class)

            if skeleton.shape[0] > num_seq:
                start = 0
                while start + num_seq < skeleton.shape[0]:
                    X.append(skeleton[start:start + num_seq])
                    Y.append(labels_pertime[start + num_seq])
                    vid_list.append(item[0])
                    start_list.append(start)
                    start = start + ovr_num
                X.append(skeleton[-num_seq:])
                Y.append(labels_pertime[-1])
                vid_list.append(item[0])
                start_list.append(skeleton.shape[0] - num_seq)
            else:
                skeleton = np.concatenate((np.zeros((num_seq - skeleton.shape[0], skeleton.shape[1], skeleton.shape[2])), skeleton), axis=0)
                labels_pertime = np.concatenate((np.zeros((num_seq - labels_pertime.shape[0], labels_pertime.shape[1])), labels_pertime), axis=0)
                X.append(skeleton)
                Y.append(labels_pertime[-1])
                vid_list.append(item[0])
                start_list.append(0)

    X = np.asarray(X).astype(np.float32)
    Y = np.asarray(Y)
    print(X.shape, Y.shape)
    return X, Y, vid_list, start_list

In [0]:
def multi_pertime(param, sub_mean=False):
    skt_input = Input(shape=(param['num_seq'], param['num_joints'], param['dim_point']))
    data = skt_input

    if sub_mean:
        data = Permute((1, 3, 2))(data)
        data2 = TimeDistributed(TimeDistributed(Dense(1, trainable=False)))(data)
        data2 = Lambda(lambda x: K.repeat_elements(x, param['num_joints'], axis=-1), output_shape=lambda s: (s[0], s[1], s[2], s[3] * param['num_joints']))(data2)
        data = Add()([data, data2])

    data = Reshape((param['num_seq'], param['num_joints'] * param['dim_point']))(data)

    hid_size = 256
    data = SpatialDropout1D(0.05)(data)
    out = Bidirectional(LSTM(hid_size, return_sequences=True))(data)
    #out = SpatialDropout1D(0.05)(out)
    out = Bidirectional(LSTM(hid_size, return_sequences=True))(out)
    #out = SpatialDropout1D(0.05)(out)
    out = Bidirectional(LSTM(hid_size, return_sequences=False))(out)

    # out = BatchNormalization()(out)
    out = Activation('relu')(Dropout(0.5)(out))
    prob = Dense(param['num_class'], activation='softmax')(out)

    opt = SGD(lr=param['base_learn_rate'], decay=param['weight_regular'], momentum=0.9, nesterov=True)
    model = Model(skt_input, prob)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.summary()
    return model

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def schedule(epoch):
    lr = K.get_value(model.optimizer.lr)
    if epoch % param['step_inter'] == 0 and epoch > 0:
        lr = lr * param['lr_gamma']
    return np.float(lr)

In [0]:
param = {'max_iter': 500, 'step_inter': 80, 'base_learn_rate': 0.01, 'lr_gamma': 0.5, 'weight_regular': 0, 'num_seq': 50, 'batchsize': 1024, 'write_file': True,
         'num_joints': 25, 'num_class': 51 + 1, 'dim_point': 3 * 2, 'save_model': True}
param['trn_file'] = F"/content/drive/My Drive/crs_view_trn_skt"
param['val_file'] = F"/content/drive/My Drive/crs_view_val_skt"
param['save_path'] = F"/content/drive/My Drive/save_param_temp/gcn/weights.hdf5"
param['write_file_name'] = F"/content/drive/My Drive/save_param_temp/gcn.txt"
# no batch size, as one batch only

trainX, trainY, train_vid_list, train_start_list = load_seq_lst(param['trn_file'], param['num_seq'])
valX, valY, val_vid_list, val_start_list = load_seq_lst(param['val_file'], param['num_seq'])

(22708, 50, 25, 6) (22708, 52)
(12767, 50, 25, 6) (12767, 52)


In [0]:
model = multi_pertime(param, sub_mean=True)
reduce_lr = LearningRateScheduler(schedule)
checkpoint = ModelCheckpoint(param['save_path'], verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=5)

model.fit(trainX, trainY, batch_size=param['batchsize'], epochs=param['max_iter'], shuffle=True, validation_data=(valX, valY),callbacks=[checkpoint], verbose=1)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 50, 25, 6)    0                                            
__________________________________________________________________________________________________
permute_3 (Permute)             (None, 50, 6, 25)    0           input_3[0][0]                    
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 50, 6, 1)     26          permute_3[0][0]                  
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 50, 6, 25)    0           time_distributed_6[0][0]         
____________________________________________________________________________________________